In [1]:
!pip install -q alpaca-py python-dotenv pandas numpy matplotlib

In [2]:
import os
from dotenv import load_dotenv
from alpaca.trading.client import TradingClient
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import time
import numpy as np

In [3]:
# Uncomment if on Google Colab
# from google.colab import userdata
# API_KEY = userdata.get('API_KEY')
# SECRET_KEY = userdata.get('SECRET_KEY')

# For local use
load_dotenv()
API_KEY = os.getenv('API_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')

In [4]:
# Connect to Alpaca Trading API
trading_client = TradingClient(API_KEY, SECRET_KEY, paper=True)
account = trading_client.get_account()
print(f"Connection successful. Account status: {account.status}")

Connection successful. Account status: AccountStatus.ACTIVE


In [5]:
# Connect to Alpaca Market Data API and create request
data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)

request_params = StockBarsRequest(
    symbol_or_symbols=["SPY"],
    timeframe=TimeFrame.Day,
    start="2022-01-01"
)

bars = data_client.get_stock_bars(request_params)
data = bars.df
print("Successfully fetched historical data for SPY:\n")
print(data.tail())

Successfully fetched historical data for SPY:

                                    open      high      low   close  \
symbol timestamp                                                      
SPY    2025-09-23 04:00:00+00:00  666.72  667.3400  661.980  663.21   
       2025-09-24 04:00:00+00:00  664.51  664.6100  659.670  661.10   
       2025-09-25 04:00:00+00:00  657.94  659.4056  654.405  658.05   
       2025-09-26 04:00:00+00:00  659.51  662.3700  657.880  661.82   
       2025-09-29 04:00:00+00:00  664.36  665.2800  661.860  663.68   

                                      volume  trade_count        vwap  
symbol timestamp                                                       
SPY    2025-09-23 04:00:00+00:00  81708872.0     896647.0  664.434604  
       2025-09-24 04:00:00+00:00  68082229.0     807609.0  661.597947  
       2025-09-25 04:00:00+00:00  89622067.0     935968.0  657.338549  
       2025-09-26 04:00:00+00:00  69179209.0     769909.0  660.616791  
       2025-09-29 04:00

In [ ]:
while True: 
    print(f"Checking for signals at {pd.Timestamp.now(tz='America/Chicago').isoformat()} ---")

    # 1. FETCH DATA (already done in the previous step, assuming 'data' DataFrame exists)
    
    # 2. ANALYZE DATA
    data['SMA50'] = data['close'].rolling(window=50).mean()
    data['Position'] = np.where(data['close'] > data['SMA50'], 1, 0)
    data['Signal'] = data['Position'].diff()
    
    
    # Let's look at the most recent data point
    latest_close = data['close'].iloc[-1]
    latest_signal = data['Signal'].iloc[-1]

    print(f"Latest Close: {latest_close:.2f}")
    print(f"Latest Signal: {latest_signal}")

    # 3. ACT (make trade)
    # Check current position and signal first
    # Alpaca API raises an error if you ask for a position that doesn't exist.
    try:
        position = trading_client.get_open_position('SPY')
        position_exists = True
    except Exception as e:
        position_exists = False

    # IF the signal is "buy" AND we don't have a position, THEN submit a buy order.
    if latest_signal == 1.0 and not position_exists:
        print("Buy signal detected! Submitting market order for 1 share.")
        trading_client.submit_order(
        symbol="SPY",
        qty=1,
        side="buy",
        type="market",
        time_in_force="gtc" # good-til-cancelled
    )
    # IF the signal is "sell" AND we already have a position, THEN sell it.
    elif latest_signal == -1.0 and position_exists:
        print("Sell signal detected! Selling our 1 share.")
        trading_client.submit_order(
        symbol="SPY",
        qty=1,
        side="buy",
        type="market",
        time_in_force="gtc" # good-til-cancelled
    )
    else:
        print("Signal is neutral or position status prevents action. Holding.")
    
    
    # 4. WAIT
    print("Waiting for 60 seconds before the next check...")
    time.sleep(60) # Pauses the script for 60 seconds

Checking for signals at 2025-09-29T15:38:17.759898-05:00 ---
Latest Close: 663.68
Latest Signal: 0.0
Signal is neutral or position status prevents action. Holding.
Waiting for 60 seconds before the next check...
